参考
https://qiita.com/TomokIshii/items/3729c1b9c658cc48b5cb

In [1]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=0)

In [3]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
# lightgbmデータセット型に変換
# evalをもとにラウンドの早期打ち切りとか決定するけど、汎化性能を見るためにはもう一つtestセットが必要では＞

In [4]:
# LightGBM parameters
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_logloss'},
        'num_class': 3,
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        #'num_iteration': 100, #trainのときにraund数は指定できるので
        'verbose': 0
}

In [5]:
# train
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=150,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)

[1]	valid_0's multi_logloss: 0.954369
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 0.83502
[3]	valid_0's multi_logloss: 0.734525
[4]	valid_0's multi_logloss: 0.648829
[5]	valid_0's multi_logloss: 0.57505
[6]	valid_0's multi_logloss: 0.511057
[7]	valid_0's multi_logloss: 0.45522
[8]	valid_0's multi_logloss: 0.407088
[9]	valid_0's multi_logloss: 0.364731
[10]	valid_0's multi_logloss: 0.327329
[11]	valid_0's multi_logloss: 0.294204
[12]	valid_0's multi_logloss: 0.264795
[13]	valid_0's multi_logloss: 0.238628
[14]	valid_0's multi_logloss: 0.215318
[15]	valid_0's multi_logloss: 0.194491
[16]	valid_0's multi_logloss: 0.175858
[17]	valid_0's multi_logloss: 0.159166
[18]	valid_0's multi_logloss: 0.144197
[19]	valid_0's multi_logloss: 0.130761
[20]	valid_0's multi_logloss: 0.118691
[21]	valid_0's multi_logloss: 0.10784
[22]	valid_0's multi_logloss: 0.0976998
[23]	valid_0's multi_logloss: 0.0885679
[24]	valid_0's multi_logloss: 0.0805959
[25]	valid_0

In [6]:
y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration)

In [7]:
accuracy_score(y_train, np.argmax(y_pred, axis=1)) #正答率100%とかっていう恐ろしいスコア

1.0

In [8]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [9]:
accuracy_score(y_test, np.argmax(y_pred, axis=1)) #正答率100%とかっていう恐ろしいスコア

1.0

In [10]:
print('Feature importances:', list(gbm.feature_importance()))

Feature importances: [1432, 657, 954, 606]


0つ目の特徴と2つ目の特徴が重要そう